In [ ]:
import torch


# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    
if torch.cuda.device_count() > 0:
  print("Let's use", torch.cuda.device_count(), "GPUs!")

CUDA is available!  Training on GPU ...
Let's use 1 GPUs!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install window_slider

  ERROR: Failed building wheel for window-slider
  Running setup.py clean for window-slider
Failed to build window-slider
  Running setup.py install for window-slider ... done


In [ ]:
import pandas as pd
import numpy as np
import torch
from window_slider import Slider
from torch.utils import data as D

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


df=pd.read_csv('drive/My Drive/Sensor Data/allraw.csv',index_col=0)
df=df[df.columns[~df.columns.str.contains('magnetometer')]]

rep=df['activityID'].unique()
val=[i for i in range(11)]
df['activityID'].replace(rep,val,inplace=True)


/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
def get2d(dfr,dfx,fst,scnd,height,width,overlap_count,lst):
  
  indx=np.arange(fst,scnd+1)
  slider = Slider(width,overlap_count)
  slider.fit(indx) 
   #i=0 
  while True: 
    window_data = slider.slide()
    if slider.reached_end_of_list():break
    start=window_data[0]
    end  =window_data[-1]+1        
    label=dfr['activityID'][start]        
    ####
    #check index with corresponding label
#         if(i==0):print(start,label)
#         i+=1     
    ####       
    slic=dfx[start:end:].T
    as_image=np.reshape(slic.values, (-1,height,width))   
    lst.append([as_image,label])
  return lst


In [ ]:
def getasimage(dfr,width,overlap):
  

  sub=dfr['subject'].unique()
  act=dfr['activityID'].unique()
  overlap_count = int(width*overlap)
  dfx=dfr.drop(columns=['timestamp-s','activityID','subject'])    
  Index=dfr.ne(dfr.shift()).filter(like='activityID').apply(lambda x: x.index[x].tolist())[0] 
  length=len(Index)-1
  lastval=len(dfx)-1
  high=dfx.shape[1] #18
  data=[]
  for i in range(length):
    first=Index[i]
    second=Index[i+1]
    data=get2d(dfr,dfx,first,second,high,width,overlap_count,data)

  data=get2d(dfr,dfx,second,lastval,high,width,overlap_count,data)
  return data

In [ ]:
import os
import random

def seed_torch(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_torch(1029)



In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
# from modeling.sync_batchnorm.batchnorm import SynchronizedBatchNorm2d

class _ASPPModule(nn.Module):
    def __init__(self, inplanes, planes, kernel_size, padding, dilation, BatchNorm):
        super(_ASPPModule, self).__init__()
        self.atrous_conv = nn.Conv2d(inplanes, planes, kernel_size=kernel_size,
                                            stride=1, padding=padding, dilation=dilation, bias=False)
        self.bn = BatchNorm(planes)
        self.relu = nn.ReLU()

        self._init_weight()

    def forward(self, x):
        x = self.atrous_conv(x)
        x = self.bn(x)

        return self.relu(x)

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
#             elif isinstance(m, SynchronizedBatchNorm2d):
#                 m.weight.data.fill_(1)
#                 m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

class ASPP(nn.Module):
    def __init__(self, inputs,ouputs, output_stride, BatchNorm):
        super(ASPP, self).__init__()

        inplanes = inputs
        if output_stride == 16:
            dilations = [1, 6, 12, 18]
        elif output_stride == 8:
            dilations = [1, 12, 24, 36]
        else:
            raise NotImplementedError

        self.aspp1 = _ASPPModule(inplanes, ouputs, 1, padding=0, dilation=dilations[0], BatchNorm=BatchNorm)
        self.aspp2 = _ASPPModule(inplanes, ouputs, 3, padding=dilations[1], dilation=dilations[1], BatchNorm=BatchNorm)
        self.aspp3 = _ASPPModule(inplanes, ouputs, 3, padding=dilations[2], dilation=dilations[2], BatchNorm=BatchNorm)
        self.aspp4 = _ASPPModule(inplanes, ouputs, 3, padding=dilations[3], dilation=dilations[3], BatchNorm=BatchNorm)

        self.global_avg_pool = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)),
                                             nn.Conv2d(inplanes, ouputs, 1, stride=1, bias=False),
                                             BatchNorm(ouputs),
                                             nn.ReLU())
        self.conv1 = nn.Conv2d(160, ouputs, 1, bias=False)
        self.bn1 = BatchNorm(ouputs)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self._init_weight()

    def forward(self, x):
        x1 = self.aspp1(x)
        x2 = self.aspp2(x)
        x3 = self.aspp3(x)
        x4 = self.aspp4(x)
        x5 = self.global_avg_pool(x)
        x5 = F.interpolate(x5, size=x4.size()[2:], mode='bilinear', align_corners=True)
        x = torch.cat((x1, x2, x3, x4, x5), dim=1)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        return self.dropout(x)

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                torch.nn.init.kaiming_normal_(m.weight)
#             elif isinstance(m, SynchronizedBatchNorm2d):
#                 m.weight.data.fill_(1)
#                 m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()


def build_aspp(inputs,outputs, output_stride, BatchNorm):
    return ASPP(inputs,outputs, output_stride, BatchNorm)

# inn = torch.randn(2,20,32,32)
# mode=build_aspp(20,32,8,nn.BatchNorm2d)
# out=mode(inn)
# out.shape

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 20, 3, padding=1)
        
        self.aspp = build_aspp(20, 32,output_stride=8, BatchNorm=nn.BatchNorm2d) 

        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)
        # linear layer ([20, 64, 4, 12]-> 500)
        self.fc1 = nn.Linear(3072, 500)
        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(500, 11)
        # dropout layer (p=0.25)
        self.dropoutcn = nn.Dropout(0.25)
        self._init_weight()
        

    def _init_weight(self):
        for m in self.modules():
          if type(m) == nn.Linear:
              nn.init.xavier_uniform(m.weight)
              m.bias.data.fill_(0.01)
        


    def forward(self, x):
        
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x=self.aspp(x)
        
        x = self.pool(F.relu(self.conv2(x)))
        
        # add dropout layer
        x = self.dropoutcn(x)
        
        x = x.view(-1,3072) 
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropoutcn(x)
        # add 2nd hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        
#        # this is for upsampling
#         x=F.interpolate(x, size=[30,30], mode='bilinear',align_corners=True)
        
        return F.log_softmax(x, dim=1)

# create a complete CNN

device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model = Net()
# model=model.to(device)
# print(model.fc1.weight.type())


# inn = torch.randn(20,1,18,50)
# inn=inn.to(device)
# out=model(inn)
# out



In [ ]:
# model_save_name = 'classifier.pt'
# path = F"drive/My Drive/Sensor Data/{model_save_name}" 
# # torch.save(model.state_dict(), path)
# torch.save({'state_dict': model.state_dict()}, path)

In [ ]:
def trainmodel(train_loader,n_epochs):
  
#   path = "drive/My Drive/Sensor Data/classifier.pt" 
#   the_model=torch.load(path)['state_dict']
#   model=the_model


  model = Net()
#   state_dict = torch.load(path)['state_dict']
#   model.load_state_dict(state_dict)
  model=model.to(device)
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
  model.train()

  for epoch in range(n_epochs):
    train_loss = 0.0
    for data, target in train_loader:

      data=data.float()
      target=target.long()
      data,target= data.to(device),target.to(device)
      optimizer.zero_grad()
      output = model(data)
#       loss = criterion(output, target)
      loss=F.nll_loss(output, target)
      loss.backward()
      optimizer.step()
      
      train_loss+=loss.item()*data.size(0)
      
    train_loss = train_loss/len(train_loader.sampler)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1,train_loss))
    
  path = "drive/My Drive/Sensor Data/checkpoint.pth" 
  checkpoint = {'model': Net(), 'state_dict': model.state_dict(),'optimizer' : optimizer.state_dict()}
  torch.save(checkpoint, path)

  


def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    model.eval()
    
    return model
 
  
def testmodel(test_loader):

  correct = 0
  total = 0
  path =  "drive/My Drive/Sensor Data/checkpoint.pth" 
  model = load_checkpoint(path)  
  
  with torch.no_grad():
      for data,target in test_loader:
        
          data=data.float()
          target=target.long()
          data,target= data.to(device),target.to(device)
          outputs = modell(data)
          _, predicted = torch.max(outputs.data, 1)
          total += target.size(0)
          correct += (predicted == target).sum().item()
  print("total {} correct {}".format(total,correct))
  print('Accuracy of the network on test data: %d %%' % (100 * correct / total))

In [ ]:
modell = TheModelClass(*args, **kwargs)

NameError: ignored

In [ ]:
from statistics import mean 
classes=['1ly1','2sit2','3stand3','4iron17','5vacClean16','6asStair12','7desStair13','8walk4','9run7','10cycl6',    
       '11nordwalk5']
# seed_torch(1029)

# seed = 0
# torch.manual_seed(seed)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed_all(seed)

criterion=nn.CrossEntropyLoss()

  
# number of subprocesses to use for data loading
num_workers = 0
batch_size = 20
epoch=5
overlp=.9
wdth=50
a=df['subject'].unique()
l=len(a)
acc=[]

def resind(df):
  df.reset_index(drop=True,inplace=True)
  return df

for i in range(l):
  
  trains=df[(df.subject != a[i])]
  tests=df.loc[df['subject'].isin([a[i]])]
  
  trains=resind(trains)
  tests=resind(tests)

  train_data=getasimage(trains,wdth,overlp)

  test_data=getasimage(tests,wdth,overlp)

  train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,num_workers=num_workers)
  test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,num_workers=num_workers)

  m=trainmodel(train_loader,epoch)
  testmodel(test_loader,m)
  
#   print("subject no. {} has accuracy {}".format(a[i],accuracy))
#   acc.append(accuracy)
  print('\n')  
  
# print('\n')  
# print("***********")
# print('\n')
# print('Average accuracy: ',mean(acc))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


Epoch: 1 	Training Loss: 0.085830
Epoch: 2 	Training Loss: 0.046212
Epoch: 3 	Training Loss: 0.044621
Epoch: 4 	Training Loss: 0.040010
Epoch: 5 	Training Loss: 0.036947
total 35801 correct 10439
Accuracy of the network on test data: 29 %


Epoch: 1 	Training Loss: 0.137411
Epoch: 2 	Training Loss: 0.107432
Epoch: 3 	Training Loss: 0.120296
Epoch: 4 	Training Loss: 0.114072
Epoch: 5 	Training Loss: 0.111004
total 38979 correct 11116
Accuracy of the network on test data: 28 %


Epoch: 1 	Training Loss: 0.159676
Epoch: 2 	Training Loss: 0.137562
Epoch: 3 	Training Loss: 0.123161
Epoch: 4 	Training Loss: 0.122594
Epoch: 5 	Training Loss: 0.113291
total 39608 correct 4826
Accuracy of the network on test data: 12 %


Epoch: 1 	Training Loss: 0.103770
Epoch: 2 	Training Loss: 0.073533
Epoch: 3 	Training Loss: 0.062485
Epoch: 4 	Training Loss: 0.046425
Epoch: 5 	Training Loss: 0.037630
total 43017 correct 5701
Accuracy of the network on test data: 13 %


Epoch: 1 	Training Loss: 0.091547
Epoc

In [ ]:
import time
import datetime

start_time = time.time()

criterion=nn.CrossEntropyLoss()


# prep model for training



trains=df[(df.subject != 107)]
tests=df.loc[df['subject'].isin([107])]

trains=resind(trains)
tests=resind(tests)

train_data=getasimage(trains,wdth,overlp)

test_data=getasimage(tests,wdth,overlp)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,num_workers=num_workers)

n_epochs = 10
model=Net()
model=model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
model.train() 

for epoch in range(n_epochs):
  train_loss = 0.0
  for data, target in train_loader:

    data=data.float()
    target=target.long()
    data,target= data.to(device),target.to(device)
    optimizer.zero_grad()
    output = model(data)
#       loss = criterion(output, target)
    loss=F.nll_loss(output, target)
    loss.backward()
    optimizer.step()

    train_loss+=loss.item()*data.size(0)

  train_loss = train_loss/len(train_loader.sampler)

  print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1,train_loss))

print(" ")
b=(time.time() - start_time)
print("Running time= " , str(datetime.timedelta(seconds=b)))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


Net(
  (conv1): Conv2d(1, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (aspp): ASPP(
    (aspp1): _ASPPModule(
      (atrous_conv): Conv2d(20, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (aspp2): _ASPPModule(
      (atrous_conv): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(12, 12), dilation=(12, 12), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (aspp3): _ASPPModule(
      (atrous_conv): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(24, 24), dilation=(24, 24), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (aspp4): _ASPPModule(
      (atrous_conv): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(36, 36), dilation=(36, 36), bias=False)

Epoch: 1 	Training Loss: 0.103184
Epoch: 2 	Training Loss: 0.072800
Epoch: 3 	Training Loss: 0.056965
Epoch: 4 	Training Loss: 0.049419
Epoch: 5 	Training Loss: 0.045113


KeyboardInterrupt: ignored

In [ ]:
correct = 0
total = 0
modell=model
modell.eval()
with torch.no_grad():
    for data,target in test_loader:

        data=data.float()
        target=target.long()
        data,target= data.to(device),target.to(device)
        outputs = modell(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
print("total {} correct {}".format(total,correct))
print('Accuracy of the network on test data: %d %%' % (
    100 * correct / total))

Net(
  (conv1): Conv2d(1, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (aspp): ASPP(
    (aspp1): _ASPPModule(
      (atrous_conv): Conv2d(20, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (aspp2): _ASPPModule(
      (atrous_conv): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(12, 12), dilation=(12, 12), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (aspp3): _ASPPModule(
      (atrous_conv): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(24, 24), dilation=(24, 24), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (aspp4): _ASPPModule(
      (atrous_conv): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(36, 36), dilation=(36, 36), bias=False)

total 42392 correct 13540
Accuracy of the network on test data: 31 %


In [ ]:
df['subject'].unique()

array([101, 102, 105, 106, 107, 108])

In [ ]:
for data, target in test_loader:
  print(target)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0,